In [1]:
from keras.models import Model, load_model


MIN_MATCH_COUNT = 19
import numpy as np
import cv2
import time

In [27]:
#завантажити нейромережу
model = load_model('Network')
IMG_SIZE = 128

In [15]:
#вставляє наступний кадр з трейлеру до нашого відео 
def insertFrame(frame, good, kp1, kp2, cap, img):
    ret, video_frame = cap.read()
    height, width, _ = img.shape
    video_frame = cv2.resize(video_frame, (width, height))
    #print(1)                                
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()

    h,w,c = img.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)
    #frame = cv2.polylines(frame,[np.int32(dst)],True,255,3, cv2.LINE_AA)
    imgWarp = cv2.warpPerspective(
        video_frame, M, (frame.shape[1], frame.shape[0])
    )
                    
    maskNew = np.zeros((frame.shape[0], frame.shape[1]), np.uint8)
    cv2.fillPoly(maskNew, [np.int32(dst)], (255, 255, 255))
    maskInv = cv2.bitwise_not(maskNew)
    imgAug = cv2.bitwise_and(frame, frame, mask=maskInv)
    imgAug = cv2.bitwise_or(imgWarp, imgAug)
    return imgAug

In [46]:
#проходить нейромережею та дескриптором по відео,
#розпізнає обкладинки книг Гаррі Поттера та Хоббіта (або Гобіта, кому як більше подобається)
#та заміняє їх трейлерами відповідних їм фільмів
cap = cv2.VideoCapture("data/video2.mp4")
capHP = cv2.VideoCapture("data/HarryPotter/Trailer.mp4")
capH = cv2.VideoCapture("data/Hobbit/Trailer.mp4")
capT = [capHP, capH]

img1 = cv2.imread("data/HarryPotter/0.jpg")
img2 = cv2.imread("data/Hobbit/0.jpg")
img2 = cv2.resize(img2, (0,0), fx=0.7, fy=0.7)
surf = cv2.xfeatures2d.SURF_create()
bf = cv2.BFMatcher()

kp_1, des_1 = surf.detectAndCompute(img1, None)
kp_2, des_2 = surf.detectAndCompute(img2, None)
descr = [(kp_1, des_1, 35), (kp_2, des_2, 35)]

images =[img1, img2] 
count = 0
#out = cv2.VideoWriter('result4.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 20, (640, 480))
out = cv2.VideoWriter('result2.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 20, (1280, 720))
while(True):
    reverse = False
    count += 1
    ret, frame = cap.read()
    if ret == True:
        x = cv2.resize(frame, (IMG_SIZE, IMG_SIZE), interpolation = cv2.INTER_AREA)
        x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
        x = np.expand_dims(x, axis=0)
        x = x/255.0
        p = model.predict(x)[0]#аналіз кадру нейромережею
        #font = cv2.FONT_HERSHEY_SIMPLEX
        #cv2.putText(frame, f"{p}", (10,50), font, 2, (0, 255, 0), 2, cv2.LINE_AA)
        
        if p[1] < p[0]:#визначає у випадку перетину книг, яка книга ближче до камери
            p = list(reversed(p))
            images = list(reversed(images))
            descr = list(reversed(descr))
            capT = list(reversed(capT))
            reverse = True
        
        for i in range(2):#чи присутні книги
            if p[i] < 0.2:#відсіюваємо варіант, тобто не будемо запускати дескриптор, якщо нейромережа нічого не знайшла
                continue
            try:
                kp1, des1, MIN_MATCH_COUNT = descr[i]

                kp2, des2 = surf.detectAndCompute(frame, None)
                

                good = []
                if isinstance(des2, np.ndarray):
                    matches = bf.knnMatch(des1, des2, k=2)
                    for m,n in matches:
                        if m.distance < 0.75*n.distance:
                            good.append(m)

                if (len(good)>=MIN_MATCH_COUNT):                  
                    frame = insertFrame(frame, good, kp1, kp2, capT[i], images[i])
            except BaseException:
                print("surf error")
        
        out.write(frame)

        cv2.imshow("result", frame)
        if reverse:
            images = list(reversed(images))
            descr = list(reversed(descr))
            capT = list(reversed(capT))
    else:
        break

    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

out.release()
cap.release()
capH.release()
capHP.release()
cv2.destroyAllWindows()

In [42]:
frame.shape

(480, 640, 3)